In [16]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix

# Caminho do dataset
dataset_path = "PlantVillage"

# Gerador de imagens com aumento de dados (Data Augmentation)
datagen = ImageDataGenerator(
    rescale=1.0/255,  # Normaliza os pixels para [0,1]
    rotation_range=90,  # Rotação aleatória de até 30 graus
    width_shift_range=0.2,  # Mudança horizontal
    height_shift_range=0.2,  # Mudança vertical
    shear_range=0.2,  # Transformação de cisalhamento
    zoom_range=0.2,  # Zoom aleatório
    horizontal_flip=True,  # Espelhamento horizontal
    fill_mode="nearest",  # Preenchimento das áreas alteradas
    brightness_range=[0.7, 1.3],  # Alteração de brilho
    validation_split=0.2  # 80% treino, 20% validação
)

# Carregar dados de treinamento e validação
train_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='validation',
    shuffle=False  # Mantemos as amostras ordenadas para métricas posteriores
)

# Criando a CNN
model = keras.Sequential([
    # Primeira camada convolucional
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D((2, 2)),

    # Segunda camada convolucional
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D((2, 2)),

    # Terceira camada convolucional
    keras.layers.Conv2D(128, (3, 3), activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D((2, 2)),

    # Camada totalmente conectada (Densa)
    keras.layers.Flatten(),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dropout(0.5),  # Reduz overfitting
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.5),  # Reduz overfitting

    # Camada de saída com softmax para classificação
    keras.layers.Dense(len(train_generator.class_indices), activation='softmax')
])

# Compilar o modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Callback para salvar apenas o melhor modelo
checkpoint = ModelCheckpoint("best_model.h5", monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)

# Treinamento do modelo
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=50,
    callbacks=[checkpoint],
)

# Nome das classes
class_names = list(val_generator.class_indices.keys())

# Obter rótulos verdadeiros e previsões do modelo
y_true = val_generator.classes
y_pred_probs = model.predict(val_generator)
y_pred = np.argmax(y_pred_probs, axis=1)

# Calcular matriz de confusão
conf_matrix = confusion_matrix(y_true, y_pred)

# Gerar relatório de classificação (Precision, Recall, F1-score)
report = classification_report(y_true, y_pred, target_names=class_names, output_dict=True)
df_report = pd.DataFrame(report).transpose()

# 🔹 Mostrar Matriz de Confusão
plt.figure(figsize=(12, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.xlabel("Classes Previstas")
plt.ylabel("Classes Verdadeiras")
plt.title("Matriz de Confusão")
plt.xticks(rotation=90)
plt.yticks(rotation=0)
plt.show()

Found 16516 images belonging to 15 classes.
Found 4122 images belonging to 15 classes.
Epoch 1/50
 13/517 [..............................] - ETA: 6:44 - loss: 10.1217 - accuracy: 0.1034

KeyboardInterrupt: 